In [391]:
import pandas as pd
import json 

data = pd.read_csv('data.csv')
# выделяею JSON в отдельную колонку
statuses = data["Description"].str.split("\n", expand=True)
data["status"] = statuses[0]
data["firebase"] = statuses[1]
data = data.drop(["Description"], axis=1)

# фильрую непутсые строки с JSON
firebase_json_not_null = data['firebase'].notnull()
firebase_json_not_null = data[firebase_json_not_null].iloc[1:100]

# разбираю JSON и превращаю его в колонки
firebase_response = firebase_json_not_null['firebase'].map(json.loads).apply(pd.Series)
firebase_json_not_null = pd.concat([firebase_json_not_null.drop(['firebase'], axis=1), firebase_response], axis=1)
firebase_response_result =  firebase_json_not_null["results"].apply(pd.Series)[0].apply(pd.Series)

# склеиваю разобранный JSON с изначальной таблицей
firebase_json_not_null = pd.concat([firebase_json_not_null.drop(["results"], axis=1), firebase_response_result], axis=1)
result = pd.concat([data.drop(['firebase'], axis=1), firebase_json_not_null.drop(['TransactionalId', 'status', "Id"], axis=1)], axis=1)

# обрабатываю данные к колонках

# в колонке много уникальных значений. Они мне не нужны, поэтоум беру первые 23 символа, чтобы остальось только 2 значения
result["status"] = result["status"].str[:23]
result["status"] = result["status"].str.replace(' \r', "")

# заполняю пустые значени 
result['error'] = result['error'].fillna('No error data')

# заполняю пустые значения и достаю первые 2 символа, чтобы было меньше уникальных значений 
result['message_id'] = result['message_id'].fillna('No message data')
result['message_id'] = result['message_id'].str[:2]

# заполняю пустые значения и преобразую к TRUE/FALSE
result['success'] = result['success'].fillna('0.0')
result['success'] = pd.to_numeric(result['success'], downcast='signed').astype(bool)
result['failure'] = result['failure'].fillna('1.0')
result['failure'] = pd.to_numeric(result['failure'], downcast='signed').astype(bool)

# собираю сводную таблицу
pd.pivot_table(result, index=['status', 'message_id'], columns=['error'], values='Id', aggfunc='count').fillna('0')


""" 
В итоге у меня получилось таблица из которой я руками достал данные вот так:

No token could be found	    141130
NotRegistered	            58555
Send OK	                    49905
InternalServerError	        522
ht (Странная ошибка)	    88
	                        250200

Но скорость выполнения низкая - минуты на этот файл. Хочу понять, как можно оптимизировать, чтобы обрабатывать файлиы в миллионы строк
"""

250200
